# Importando as bibliotecas

In [ ]:
import pandas as pd
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

tqdm.pandas()

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
from pymannkendall import original_test

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

Aplicando pipeline para preparação dos dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

collection = collection_filters.collection

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
# Visualizar as ultimas linhas do dataframe
df_original.tail()

In [ ]:
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df


# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

# Aplicando a função
df_original = preencher_e_converter(df_original, colunas_para_converter)

# Filtragem dos dados
df_original.drop(columns=['_id', 'diaDaSemana', 'hora', 'minutos', 'diaDoMes', 'mes', 'ano'], inplace=True)

# Criando novas colunas com o pandas
df_original['data'] = df_original['postCreatedTime'].dt.date
df_original['data'] = pd.to_datetime(df_original['data'])
df_original['mes'] = df_original['data'].dt.to_period('M')
df_original['semana'] = df_original['data'].dt.to_period('W')

In [ ]:
# Filtrar posts do ano de 2016
df_2017 = df_original[df_original['data'].dt.year == 2017]
df_2017 = df_2017[~((df_2017['data'].dt.month == 12) & (df_2017['data'].dt.year == 2017))]
df_2017 = df_2017[df_2017['data'].dt.month >= 5]
df_2017 = df_2017[df_2017['suicida'] == 3]
df_2017 = df_2017.copy()
df_2017.head()

In [ ]:
# Remover linhas onde ambas as colunas 'postMessage' e 'postStory' estão vazias ou nulas
df_2017 = df_2017.dropna(subset=['postMessage', 'postStory'], how='all')  # Remove quando ambas são NaN
df_2017 = df_2017[~((df_2017['postMessage'].str.strip() == '') & (
        df_2017['postStory'].str.strip() == ''))]  # Remove quando ambas são strings vazias

df_2017 = df_2017.copy()

In [ ]:
# Calcular a contagem de postagens por usuário
post_counts = df_2017.groupby('id_usuario').size().reset_index(name='post_count')

# Visualizar a distribuição
plt.hist(post_counts['post_count'], bins=50)
plt.xlabel('Número de Postagens')
plt.ylabel('Quantidade de Usuários')
plt.title('Distribuição de Postagens por Usuário')
plt.show()

In [ ]:
# Definir os limites superior e inferior
limite_superior = post_counts['post_count'].quantile(0.85)
limite_inferior = post_counts['post_count'].quantile(0.15)

# Filtrar usuários com alta atividade
usuarios_alta_ativ = post_counts[post_counts['post_count'] > limite_superior]['id_usuario']

# Filtrar usuários com baixa atividade
usuarios_baixa_ativ = post_counts[post_counts['post_count'] < limite_inferior]['id_usuario']

# Exibir o número de usuários filtrados
print(f'Número de usuários com alta atividade: {len(usuarios_alta_ativ)}')
print(f'Número de usuários com baixa atividade: {len(usuarios_baixa_ativ)}')

In [ ]:
df_normal = df_2017[
    ~(df_2017['id_usuario'].isin(usuarios_alta_ativ) | df_2017['id_usuario'].isin(usuarios_baixa_ativ))].copy()
norm_post_count = df_normal.groupby('id_usuario').size().reset_index(name='post_count')

df_alta_ativ = df_2017[df_2017['id_usuario'].isin(usuarios_alta_ativ)].copy()
alt_post_count = df_alta_ativ.groupby('id_usuario').size().reset_index(name='post_count')

df_baixa_ativ = df_2017[df_2017['id_usuario'].isin(usuarios_baixa_ativ)].copy()
baixa_post_count = df_baixa_ativ.groupby('id_usuario').size().reset_index(name='post_count')

In [ ]:
def plotar_distribuicao(post_count_df, titulo):
    plt.hist(post_count_df['post_count'], bins=50)
    plt.xlabel('Número de Postagens')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()


plotar_distribuicao(norm_post_count, 'Distribuição de Postagens por Usuário (Normal)')
plotar_distribuicao(alt_post_count, 'Distribuição de Postagens por Usuário (Alta Atividade)')
plotar_distribuicao(baixa_post_count, 'Distribuição de Postagens por Usuário (Baixa Atividade)')

In [ ]:
# Salvar os DataFrames em arquivos CSV
df_normal.to_csv('posts_normal.csv', index=False)
df_alta_ativ.to_csv('posts_alta_atividade.csv', index=False)
df_baixa_ativ.to_csv('posts_baixa_atividade.csv', index=False)

In [ ]:
# Contar o número de meses únicos
meses_unicos = df_normal['mes'].unique()
num_meses = len(meses_unicos)
print(f"Número de meses únicos: {num_meses}")

posts_por_usuario_mes = df_normal.groupby(['id_usuario', 'mes']).size().reset_index(name='num_posts')

# Contar o número total de meses ativos para cada usuário
meses_ativos_por_usuario = posts_por_usuario_mes.groupby('id_usuario')['mes'].nunique().reset_index(name='meses_ativos')

# Filtrar usuários que têm posts em todos os meses (pelo menos um post por mês)
usuarios_com_posts_todos_meses = meses_ativos_por_usuario[meses_ativos_por_usuario['meses_ativos'] == num_meses]

# Exibir os usuários que têm posts em todos os meses
print(usuarios_com_posts_todos_meses)

In [ ]:
# Filtrar os usuários que têm posts em todos os meses 
df_normal = df_normal[df_normal['id_usuario'].isin(usuarios_com_posts_todos_meses['id_usuario'])]

# Exibir o DataFrame filtrado
df_normal.head()

In [ ]:
posts_por_dia = df_normal.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')
posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()
posts_por_dia.head()

In [ ]:
# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Mês')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

In [ ]:
posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Semana')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

In [ ]:
periodo = 30

# Lista para armazenar DataFrames de tendências de cada usuário
lista_tendencias = []

# Iterar sobre cada usuário
for usuario in df_normal['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_normal[df_normal['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            X_sm = sm.add_constant(X)
            model_sm = sm.OLS(y, X_sm).fit()
            # print(model_sm.summary())

            r_squared = model.score(X, y)
            print(f'R² da regressão para o usuário {usuario}: {r_squared:.4f}')

            # Exibir o resultado
            print(f'Inclinação (slope) da tendência para o usuário {usuario}: {slope}')

            result = adfuller(y)
            print('Estatística ADF:', result[0])
            print('P-valor:', result[1])

            # Armazenar a tendência em um DataFrame
            df_tendencia = tendencia.reset_index()
            df_tendencia.columns = ['Data', 'Tendência']
            df_tendencia['id_usuario'] = usuario  # Adicionar coluna com o ID do usuário

            # Adicionar o DataFrame à lista
            lista_tendencias.append(df_tendencia)

            # Plotar os dados e a linha de tendência ajustada
            # plt.figure(figsize=(15, 12))
            # plt.plot(tendencia.index, tendencia.values, label='Tendência Original')
            # plt.plot(tendencia.index, model.predict(X), label='Tendência Ajustada (Regressão Linear)', linestyle='--')
            # plt.title(f'Tendência para o Usuário {usuario}')
            # plt.xlabel('Data')
            # plt.ylabel('Tendência')
            # plt.legend()
            # plt.grid(True)
            # plt.show()

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, concatenar todas as tendências em um único DataFrame
df_todas_tendencias = pd.concat(lista_tendencias, ignore_index=True)

# Exibir as primeiras linhas do DataFrame resultante
df_todas_tendencias.head()

In [ ]:
periodo = 30

# Lista para armazenar os resultados
resultados_mk = []

# Iterar sobre cada usuário
for usuario in df_normal['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_normal[df_normal['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Aplicar o teste de Mann-Kendall
            result = original_test(tendencia)
            print(result, '\n')

            # Armazenar os resultados em um dicionário
            resultados_mk.append({
                'id_usuario': usuario,
                'trend': result.trend,
                'h': result.h,
                'p-valor': result.p,
                'z': result.z,
                'Tau': result.Tau,
                's': result.s,
                'var_s': result.var_s,
                'slope': result.slope,
                'intercept': result.intercept
            })

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, converter a lista de resultados em um DataFrame
df_resultados_mk = pd.DataFrame(resultados_mk)

# Exibir as primeiras linhas do DataFrame
df_resultados_mk.head()

# Opcional: Salvar o DataFrame em um arquivo CSV
df_resultados_mk.to_csv('resultados_mann_kendall.csv', index=False)

# Contar o número de usuários em cada tipo de tendência
contagem_tendencias = df_resultados_mk['trend'].value_counts()
print(contagem_tendencias)

# Calcular percentuais
total_usuarios = df_resultados_mk.shape[0]
percentuais = (contagem_tendencias / total_usuarios) * 100
print(percentuais)

import seaborn as sns

# Gráfico de barras das tendências
plt.figure(figsize=(8, 6))
sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
plt.title('Distribuição de Tendências entre Usuários')
plt.xlabel('Tipo de Tendência')
plt.ylabel('Número de Usuários')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
plt.title('Distribuição dos Slopes entre Usuários')
plt.xlabel('Slope')
plt.ylabel('Frequência')
plt.show()

# Filtrar usuários com tendência crescente significativa
usuarios_crescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'increasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência crescente significativa: {len(usuarios_crescentes)}")

# Filtrar usuários com tendência decrescente significativa
usuarios_decrescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'decreasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência decrescente significativa: {len(usuarios_decrescentes)}")

# Filtrar usuários sem tendência significativa
usuarios_sem_tendencia = df_resultados_mk[(df_resultados_mk['trend'] == 'no trend') & (df_resultados_mk['h'] == False)]
print(f"Usuários sem tendência significativa: {len(usuarios_sem_tendencia)}")

In [ ]:
df_resultados_mk.head()

In [ ]:
df_trend_decreasing = df_resultados_mk[df_resultados_mk['trend'] == 'decreasing']
usuarios_decreasing = df_trend_decreasing['id_usuario']

# Exibir os resultados
usuarios_decreasing.head()

In [ ]:
# filtrar no dataframe df_2017 apenas os usuarios presentes no dataframe usuarios_decreasing
df_2017_decreasing = df_normal[df_normal['id_usuario'].isin(usuarios_decreasing)].copy()

In [ ]:
from utils.estracao_interacao import ExtracaoInteracao

extracao = ExtracaoInteracao(df_2017_decreasing)
df_with_interactions = extracao.extract_interactions()

# Passo 6: Modificar outras colunas usando .loc para evitar avisos
df_with_interactions['sexo'] = df_with_interactions['sexo'].map({'F': 0, 'M': 1})
df_with_interactions.head()

In [ ]:
from utils.text_treatment import TextTreatment
from utils.busca_palavras import BuscaPalavras

tratamento_texto = TextTreatment()
df_with_interactions['postMessageLimpo'] = df_with_interactions['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

busca_palavras = BuscaPalavras()

# Faz a busca exata
resultado = busca_palavras.string_matching(df_with_interactions['postMessageLimpo'],
                                           'dados/datasets/termos_depressivos_pt_br.txt')

# Adicionar a coluna quantPalavrasDepressivas ao DataFrame, contando quantas palavras depressivas foram encontradas
df_with_interactions['quantPalavrasDepressivas'] = resultado.apply(lambda x: len(x.split(', ')) if x else 0)
df_with_interactions.head()

In [ ]:
periodo = 30

# Lista para armazenar os resultados
resultados_mk = []

# Iterar sobre cada usuário
for usuario in df_with_interactions['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_with_interactions[df_with_interactions['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data')['quantProfile'].sum()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Aplicar o teste de Mann-Kendall
            result = original_test(tendencia)
            print(result, '\n')

            # Armazenar os resultados em um dicionário
            resultados_mk.append({
                'id_usuario': usuario,
                'trend': result.trend,
                'h': result.h,
                'p-valor': result.p,
                'z': result.z,
                'Tau': result.Tau,
                's': result.s,
                'var_s': result.var_s,
                'slope': result.slope,
                'intercept': result.intercept
            })

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, converter a lista de resultados em um DataFrame
df_resultados_mk = pd.DataFrame(resultados_mk)

# Exibir as primeiras linhas do DataFrame
df_resultados_mk.head()

# Opcional: Salvar o DataFrame em um arquivo CSV
df_resultados_mk.to_csv('resultados_mann_kendall.csv', index=False)

# Contar o número de usuários em cada tipo de tendência
contagem_tendencias = df_resultados_mk['trend'].value_counts()
print(contagem_tendencias)

# Calcular percentuais
total_usuarios = df_resultados_mk.shape[0]
percentuais = (contagem_tendencias / total_usuarios) * 100
print(percentuais)

import seaborn as sns

# Gráfico de barras das tendências
plt.figure(figsize=(8, 6))
sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
plt.title('Distribuição de Tendências entre Usuários')
plt.xlabel('Tipo de Tendência')
plt.ylabel('Número de Usuários')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
plt.title('Distribuição dos Slopes entre Usuários')
plt.xlabel('Slope')
plt.ylabel('Frequência')
plt.show()

# Filtrar usuários com tendência crescente significativa
usuarios_crescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'increasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência crescente significativa: {len(usuarios_crescentes)}")

# Filtrar usuários com tendência decrescente significativa
usuarios_decrescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'decreasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência decrescente significativa: {len(usuarios_decrescentes)}")

# Filtrar usuários sem tendência significativa
usuarios_sem_tendencia = df_resultados_mk[(df_resultados_mk['trend'] == 'no trend') & (df_resultados_mk['h'] == False)]
print(f"Usuários sem tendência significativa: {len(usuarios_sem_tendencia)}")

# Analisar usuarios com atividade alta

In [ ]:
# Contar o número de meses únicos
meses_unicos = df_alta_ativ['mes'].unique()
num_meses = len(meses_unicos)
print(f"Número de meses únicos: {num_meses}")

posts_por_usuario_mes = df_alta_ativ.groupby(['id_usuario', 'mes']).size().reset_index(name='num_posts')

# Contar o número total de meses ativos para cada usuário
meses_ativos_por_usuario = posts_por_usuario_mes.groupby('id_usuario')['mes'].nunique().reset_index(name='meses_ativos')

# Filtrar usuários que têm posts em todos os meses (pelo menos um post por mês)
usuarios_com_posts_todos_meses = meses_ativos_por_usuario[meses_ativos_por_usuario['meses_ativos'] == num_meses]

# Exibir os usuários que têm posts em todos os meses
print(usuarios_com_posts_todos_meses)

In [ ]:
# Filtrar os usuários que têm posts em todos os meses 
df_alta_ativ = df_alta_ativ[df_alta_ativ['id_usuario'].isin(usuarios_com_posts_todos_meses['id_usuario'])]

# Exibir o DataFrame filtrado
df_alta_ativ.head()

In [ ]:
posts_por_dia = df_alta_ativ.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')
posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()
posts_por_dia.head()

In [ ]:
# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Mês')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

In [ ]:
posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Semana')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

In [ ]:
periodo = 30

# Lista para armazenar DataFrames de tendências de cada usuário
lista_tendencias = []

# Iterar sobre cada usuário
for usuario in df_alta_ativ['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_alta_ativ[df_alta_ativ['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            X_sm = sm.add_constant(X)
            model_sm = sm.OLS(y, X_sm).fit()
            # print(model_sm.summary())

            r_squared = model.score(X, y)
            print(f'R² da regressão para o usuário {usuario}: {r_squared:.4f}')

            # Exibir o resultado
            print(f'Inclinação (slope) da tendência para o usuário {usuario}: {slope}')

            result = adfuller(y)
            print('Estatística ADF:', result[0])
            print('P-valor:', result[1])

            # Armazenar a tendência em um DataFrame
            df_tendencia = tendencia.reset_index()
            df_tendencia.columns = ['Data', 'Tendência']
            df_tendencia['id_usuario'] = usuario  # Adicionar coluna com o ID do usuário

            # Adicionar o DataFrame à lista
            lista_tendencias.append(df_tendencia)

            # Plotar os dados e a linha de tendência ajustada
            plt.figure(figsize=(15, 12))
            plt.plot(tendencia.index, tendencia.values, label='Tendência Original')
            plt.plot(tendencia.index, model.predict(X), label='Tendência Ajustada (Regressão Linear)', linestyle='--')
            plt.title(f'Tendência para o Usuário {usuario}')
            plt.xlabel('Data')
            plt.ylabel('Tendência')
            plt.legend()
            plt.grid(True)
            plt.show()

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, concatenar todas as tendências em um único DataFrame
df_todas_tendencias = pd.concat(lista_tendencias, ignore_index=True)

# Exibir as primeiras linhas do DataFrame resultante
df_todas_tendencias.head()

In [ ]:
periodo = 30

# Lista para armazenar os resultados
resultados_mk = []

# Iterar sobre cada usuário
for usuario in df_alta_ativ['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_alta_ativ[df_alta_ativ['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Aplicar o teste de Mann-Kendall
            result = original_test(tendencia)
            print(result, '\n')

            # Armazenar os resultados em um dicionário
            resultados_mk.append({
                'id_usuario': usuario,
                'trend': result.trend,
                'h': result.h,
                'p-valor': result.p,
                'z': result.z,
                'Tau': result.Tau,
                's': result.s,
                'var_s': result.var_s,
                'slope': result.slope,
                'intercept': result.intercept
            })

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, converter a lista de resultados em um DataFrame
df_resultados_mk = pd.DataFrame(resultados_mk)

# Exibir as primeiras linhas do DataFrame
df_resultados_mk.head()

# Opcional: Salvar o DataFrame em um arquivo CSV
df_resultados_mk.to_csv('resultados_mann_kendall.csv', index=False)

# Contar o número de usuários em cada tipo de tendência
contagem_tendencias = df_resultados_mk['trend'].value_counts()
print(contagem_tendencias)

# Calcular percentuais
total_usuarios = df_resultados_mk.shape[0]
percentuais = (contagem_tendencias / total_usuarios) * 100
print(percentuais)

import seaborn as sns

# Gráfico de barras das tendências
plt.figure(figsize=(8, 6))
sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
plt.title('Distribuição de Tendências entre Usuários')
plt.xlabel('Tipo de Tendência')
plt.ylabel('Número de Usuários')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
plt.title('Distribuição dos Slopes entre Usuários')
plt.xlabel('Slope')
plt.ylabel('Frequência')
plt.show()

# Filtrar usuários com tendência crescente significativa
usuarios_crescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'increasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência crescente significativa: {len(usuarios_crescentes)}")

# Filtrar usuários com tendência decrescente significativa
usuarios_decrescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'decreasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência decrescente significativa: {len(usuarios_decrescentes)}")

# Filtrar usuários sem tendência significativa
usuarios_sem_tendencia = df_resultados_mk[(df_resultados_mk['trend'] == 'no trend') & (df_resultados_mk['h'] == False)]
print(f"Usuários sem tendência significativa: {len(usuarios_sem_tendencia)}")